In [1]:
import pandas as pd
import numpy as np
import math
import operator
from itertools import combinations

In [2]:
def subsets(s):
    sets = []
    for i in range(1 << len(s)):
        subset = [s[bit] for bit in range(len(s)) if is_bit_set(i, bit)]
        sets.append(subset)
    return sets

def halfsubsets(s):
    sets = []
    for i in range(1 << len(s)):
        subset = [s[bit] for bit in range(len(s)) if is_bit_set(i, bit)]
        if(len(subset) <= len(s)/2):
            sets.append(subset)
    return sets

def is_bit_set(num, bit):
    return num & (1 << bit) > 0

# Load CSV Data

In [3]:
df = pd.read_csv('TestSample_100_XTable.csv')
del df['Unnamed: 0']
df.head(n = 20)

,ID,Cont1,Cont2,Cont3,Ord1,Ord2,Ord3,Nom1,Nom2,Nom3,A,Y
0,1,0,1,7,4,2,0,4,3,3,0,31.2112
1,2,9,0,3,0,3,3,2,3,3,0,20.8458
2,3,2,7,0,1,3,3,4,1,3,0,94.7321
3,4,2,3,0,0,0,2,3,2,0,1,22.4179
4,5,4,4,2,2,3,4,1,4,1,1,15.0723
5,6,9,4,1,4,0,2,2,3,4,1,21.5173
6,7,1,2,9,3,1,2,2,0,4,0,29.9679
7,8,1,8,2,3,1,0,4,3,3,0,0.4875
8,9,0,2,8,0,2,0,1,4,1,1,63.6878
9,10,8,4,3,4,1,4,1,4,2,1,19.1140


## Step 1. Create cut info 

In [4]:
unique_X0 = sorted((df['Cont1']+1).unique())
unique_X1 = sorted((df['Cont2']+1).unique())
unique_X2 = sorted((df['Cont3']+1).unique())
unique_X3 = sorted((df['Ord1']+1).unique())
unique_X4 = sorted((df['Ord2']+1).unique())
unique_X5 = sorted((df['Ord3']+1).unique())
unique_X6 = sorted(df['Nom1'].unique())
sets_X6 = halfsubsets(unique_X6)
unique_X7 = sorted(df['Nom2'].unique())
sets_X7 = halfsubsets(unique_X7)
unique_X8 = sorted(df['Nom3'].unique())
sets_X8 = halfsubsets(unique_X8)

In [5]:
var_dict = {0:10,1:10,2:10,3:5,4:5,5:5,6:16,7:16,8:16}
var = [0,1,2,3,4,5,6,7,8]
var_comb = list(combinations(var,2))
var_info = pd.DataFrame(var_comb)
var_info.columns = ['var i', 'var j']
var_info['N'] = var_info['var i'].map(var_dict) * var_info['var j'].map(var_dict)
var_info['N'].sum(axis = 0)

3753

In [6]:
var_info['cum_N'] = var_info['N'].cumsum()
var_info

,var i,var j,N,cum_N
0,0,1,100,100
1,0,2,100,200
2,0,3,50,250
3,0,4,50,300
4,0,5,50,350
5,0,6,160,510
6,0,7,160,670
7,0,8,160,830
8,1,2,100,930
9,1,3,50,980


## Step 2. Search

In [7]:
X_Search = df.copy()[['Cont1','Cont2','A','Y']];
X_Search['C1'] = df['Cont1'].apply(lambda x : 1 if x<2 else 0)
X_Search['C2'] = df['Cont2'].apply(lambda x : 1 if x<3 else 0)
X_Search['D1'] = X_Search.apply(lambda row: 1 if (row['C1']==1)&(row['C2']==1) else 0, axis=1)
X_Search['D2'] = X_Search.apply(lambda row: 1 if (row['C1']==0)&(row['C2']==1) else 0, axis=1)
X_Search['D3'] = X_Search.apply(lambda row: 1 if (row['C1']==1)&(row['C2']==0) else 0, axis=1)
X_Search['D4'] = X_Search.apply(lambda row: 1 if (row['C1']==0)&(row['C2']==0) else 0, axis=1)

In [8]:
X_Search[['D1','D2','D3','D4']].sum(axis=0)

D1     8
D2    22
D3    12
D4    58
dtype: int64

In [9]:
sum(X_Search[['D1','D2','D3','D4']].sum(axis=0))

100

In [10]:
X_Search['E1'] = np.where(X_Search['D1']==X_Search['A'], 1, 0)
X_Search['E2'] = np.where(X_Search['D2']==X_Search['A'], 1, 0)
X_Search['E3'] = np.where(X_Search['D3']==X_Search['A'], 1, 0)
X_Search['E4'] = np.where(X_Search['D4']==X_Search['A'], 1, 0)

In [11]:
E_X_Search_1 = sum(X_Search['Y']*X_Search['E1'])*2/df.shape[0]
E_X_Search_2 = sum(X_Search['Y']*X_Search['E2'])*2/df.shape[0]
E_X_Search_3 = sum(X_Search['Y']*X_Search['E3'])*2/df.shape[0]
E_X_Search_4 = sum(X_Search['Y']*X_Search['E4'])*2/df.shape[0]
E = [E_X_Search_4,E_X_Search_3,E_X_Search_2,E_X_Search_1]
E

[36.014317999999982,
 54.078689400000002,
 59.097565400000001,
 55.438553400000004]

In [12]:
index, value = max(enumerate(E), key=operator.itemgetter(1))
print("index: "+str(index))
print("max: "+str(value))

index: 2
max: 59.0975654


### Case 1. var0 and var1 (Cont 1 and Cont 2)

In [13]:
for i in unique_X0:
    for j in unique_X1: 
        X_Search = df.copy()[['Cont1','Cont2','A','Y']];
        X_Search['C1'] = df['Cont1'].apply(lambda x : 1 if x<i else 0)
        X_Search['C2'] = df['Cont2'].apply(lambda x : 1 if x<j else 0)
        X_Search['D1'] = X_Search.apply(lambda row: 1 if (row['C1']==1)&(row['C2']==1) else 0, axis=1)
        X_Search['D2'] = X_Search.apply(lambda row: 1 if (row['C1']==1)&(row['C2']==0) else 0, axis=1)
        X_Search['D3'] = X_Search.apply(lambda row: 1 if (row['C1']==0)&(row['C2']==1) else 0, axis=1)
        X_Search['D4'] = X_Search.apply(lambda row: 1 if (row['C1']==0)&(row['C2']==0) else 0, axis=1)
        X_Search['E1'] = np.where(X_Search['D1']==X_Search['A'], 1, 0)
        X_Search['E2'] = np.where(X_Search['D2']==X_Search['A'], 1, 0)
        X_Search['E3'] = np.where(X_Search['D3']==X_Search['A'], 1, 0)
        X_Search['E4'] = np.where(X_Search['D4']==X_Search['A'], 1, 0)
        E_X_Search_1 = sum(X_Search['Y']*X_Search['E1'])*2/df.shape[0]
        E_X_Search_2 = sum(X_Search['Y']*X_Search['E2'])*2/df.shape[0]
        E_X_Search_3 = sum(X_Search['Y']*X_Search['E3'])*2/df.shape[0]
        E_X_Search_4 = sum(X_Search['Y']*X_Search['E4'])*2/df.shape[0]
        E = [E_X_Search_4,E_X_Search_3,E_X_Search_2,E_X_Search_1]
        index, value = max(enumerate(E), key=operator.itemgetter(1))
        print("x0 "+str(i)+" x1 "+str(j),value,index)

x0 1 x1 1 57.8523594 3
x0 1 x1 2 61.2139794 1
x0 1 x1 3 58.5781594 1
x0 1 x1 4 60.187656 1
x0 1 x1 5 60.829382 1
x0 1 x1 6 57.2907834 3
x0 1 x1 7 56.2250214 3
x0 1 x1 8 57.3404674 2
x0 1 x1 9 57.3404674 2
x0 1 x1 10 56.8355874 0
x0 2 x1 1 57.8523594 3
x0 2 x1 2 60.7741974 1
x0 2 x1 3 59.0975654 1
x0 2 x1 4 60.707062 1
x0 2 x1 5 61.085208 1
x0 2 x1 6 56.1222674 3
x0 2 x1 7 55.0694094 2
x0 2 x1 8 56.2315814 2
x0 2 x1 9 57.3404674 2
x0 2 x1 10 56.8355874 0
x0 3 x1 1 59.7023954 3
x0 3 x1 2 60.8270694 1
x0 3 x1 3 59.1504374 1
x0 3 x1 4 60.311576 1
x0 3 x1 5 61.766306 1
x0 3 x1 6 57.048886 1
x0 3 x1 7 54.0383434 2
x0 3 x1 8 57.0123194 2
x0 3 x1 9 57.3404674 2
x0 3 x1 10 56.8355874 0
x0 4 x1 1 59.7023954 3
x0 4 x1 2 60.8270694 1
x0 4 x1 3 58.5435314 1
x0 4 x1 4 59.52892 1
x0 4 x1 5 60.98365 1
x0 4 x1 6 57.581448 1
x0 4 x1 7 52.797326 1
x0 4 x1 8 56.0443294 2
x0 4 x1 9 56.1456654 2
x0 4 x1 10 56.8355874 0
x0 5 x1 1 59.7023954 3
x0 5 x1 2 58.2965474 3
x0 5 x1 3 58.7064254 3
x0 5 x1 4 60.6633574

### Case 2. var4 and var5 (Ord 2 and Ord 3)

In [14]:
for i in unique_X4:
    for j in unique_X5: 
        X_Search = df.copy()[['Ord2','Ord3','A','Y']];
        X_Search['C1'] = df['Ord2'].apply(lambda x : 1 if x<i else 0)
        X_Search['C2'] = df['Ord3'].apply(lambda x : 1 if x<j else 0)
        X_Search['D1'] = X_Search.apply(lambda row: 1 if (row['C1']==1)&(row['C2']==1) else 0, axis=1)
        X_Search['D2'] = X_Search.apply(lambda row: 1 if (row['C1']==1)&(row['C2']==0) else 0, axis=1)
        X_Search['D3'] = X_Search.apply(lambda row: 1 if (row['C1']==0)&(row['C2']==1) else 0, axis=1)
        X_Search['D4'] = X_Search.apply(lambda row: 1 if (row['C1']==0)&(row['C2']==0) else 0, axis=1)
        X_Search['E1'] = np.where(X_Search['D1']==X_Search['A'], 1, 0)
        X_Search['E2'] = np.where(X_Search['D2']==X_Search['A'], 1, 0)
        X_Search['E3'] = np.where(X_Search['D3']==X_Search['A'], 1, 0)
        X_Search['E4'] = np.where(X_Search['D4']==X_Search['A'], 1, 0)
        E_X_Search_1 = sum(X_Search['Y']*X_Search['E1'])*2/df.shape[0]
        E_X_Search_2 = sum(X_Search['Y']*X_Search['E2'])*2/df.shape[0]
        E_X_Search_3 = sum(X_Search['Y']*X_Search['E3'])*2/df.shape[0]
        E_X_Search_4 = sum(X_Search['Y']*X_Search['E4'])*2/df.shape[0]
        E = [E_X_Search_4,E_X_Search_3,E_X_Search_2,E_X_Search_1]
        index, value = max(enumerate(E), key=operator.itemgetter(1))
        print("x4 "+str(i)+" x5 "+str(j),value,index)

x4 1 x5 1 57.5749274 3
x4 1 x5 2 56.3947694 3
x4 1 x5 3 56.1998234 3
x4 1 x5 4 55.5489314 2
x4 1 x5 5 56.8355874 0
x4 2 x5 1 57.6864274 1
x4 2 x5 2 55.7273114 3
x4 2 x5 3 53.6089794 3
x4 2 x5 4 53.4855594 2
x4 2 x5 5 56.8355874 0
x4 3 x5 1 58.5825574 1
x4 3 x5 2 53.0363934 0
x4 3 x5 3 54.1122574 1
x4 3 x5 4 54.3839054 0
x4 3 x5 5 56.8355874 0
x4 4 x5 1 58.7436554 1
x4 4 x5 2 57.2675194 0
x4 4 x5 3 57.9435474 1
x4 4 x5 4 57.3919594 1
x4 4 x5 5 56.8355874 0
x4 5 x5 1 58.0366794 3
x4 5 x5 2 56.8355874 0
x4 5 x5 3 56.8355874 0
x4 5 x5 4 56.8355874 0
x4 5 x5 5 56.8355874 0


### Case 3. var6 and var8 (Nom 1 and Nom 3)

In [15]:
for i in range(len(sets_X6)):
    for j in range(len(sets_X8)):
        X_Search = df.copy()[['Nom1','Nom3','A','Y']];
        X_Search['C1'] = df['Nom1'].apply(lambda x : 1 if x in sets_X6[i] else 0)
        X_Search['C2'] = df['Nom3'].apply(lambda x : 1 if x in sets_X8[j] else 0)
        X_Search['D1'] = X_Search.apply(lambda row: 1 if (row['C1']==1)&(row['C2']==1) else 0, axis=1)
        X_Search['D2'] = X_Search.apply(lambda row: 1 if (row['C1']==1)&(row['C2']==0) else 0, axis=1)
        X_Search['D3'] = X_Search.apply(lambda row: 1 if (row['C1']==0)&(row['C2']==1) else 0, axis=1)
        X_Search['D4'] = X_Search.apply(lambda row: 1 if (row['C1']==0)&(row['C2']==0) else 0, axis=1)
        X_Search['E1'] = np.where(X_Search['D1']==X_Search['A'], 1, 0)
        X_Search['E2'] = np.where(X_Search['D2']==X_Search['A'], 1, 0)
        X_Search['E3'] = np.where(X_Search['D3']==X_Search['A'], 1, 0)
        X_Search['E4'] = np.where(X_Search['D4']==X_Search['A'], 1, 0)
        E_X_Search_1 = sum(X_Search['Y']*X_Search['E1'])*2/df.shape[0]
        E_X_Search_2 = sum(X_Search['Y']*X_Search['E2'])*2/df.shape[0]
        E_X_Search_3 = sum(X_Search['Y']*X_Search['E3'])*2/df.shape[0]
        E_X_Search_4 = sum(X_Search['Y']*X_Search['E4'])*2/df.shape[0]
        E = [E_X_Search_4,E_X_Search_3,E_X_Search_2,E_X_Search_1]
        index, value = max(enumerate(E), key=operator.itemgetter(1))
        print("x6 "+str(i)+" x8 "+str(j),value,index)

x6 0 x8 0 56.8355874 1
x6 0 x8 1 56.8355874 2
x6 0 x8 2 58.6787314 1
x6 0 x8 3 56.8355874 2
x6 0 x8 4 56.8355874 2
x6 0 x8 5 56.8355874 2
x6 0 x8 6 56.8355874 2
x6 0 x8 7 56.8355874 2
x6 0 x8 8 56.8355874 2
x6 0 x8 9 56.8355874 2
x6 0 x8 10 56.8355874 2
x6 0 x8 11 56.8355874 2
x6 0 x8 12 56.8355874 2
x6 0 x8 13 56.8355874 2
x6 0 x8 14 56.8355874 2
x6 0 x8 15 56.8355874 2
x6 1 x8 0 56.8355874 1
x6 1 x8 1 54.3535774 3
x6 1 x8 2 62.3569314 1
x6 1 x8 3 58.9609434 1
x6 1 x8 4 58.0754934 3
x6 1 x8 5 55.5934834 3
x6 1 x8 6 54.3972934 3
x6 1 x8 7 55.4286174 3
x6 1 x8 8 53.7085874 2
x6 1 x8 9 59.4715874 1
x6 1 x8 10 56.6685234 3
x6 1 x8 11 56.1468814 3
x6 1 x8 12 53.6648714 3
x6 1 x8 13 58.3786294 1
x6 1 x8 14 57.3867874 3
x6 1 x8 15 54.7399114 3
x6 2 x8 0 60.2639354 2
x6 2 x8 1 59.0928394 3
x6 2 x8 2 59.6342234 2
x6 2 x8 3 59.7225514 3
x6 2 x8 4 59.6708554 2
x6 2 x8 5 59.6859194 3
x6 2 x8 6 59.0411434 2
x6 2 x8 7 60.3156314 2
x6 2 x8 8 59.0411434 3
x6 2 x8 9 59.6859194 2
x6 2 x8 10 59.7225514 

### Case 4. var0 and var3 (Cont 1 and Ord 1)

In [16]:
for i in unique_X0:
    for j in unique_X3: 
        X_Search = df.copy()[['Cont1','Ord1','A','Y']];
        X_Search['C1'] = df['Cont1'].apply(lambda x : 1 if x<i else 0)
        X_Search['C2'] = df['Ord1'].apply(lambda x : 1 if x<j else 0)
        X_Search['D1'] = X_Search.apply(lambda row: 1 if (row['C1']==1)&(row['C2']==1) else 0, axis=1)
        X_Search['D2'] = X_Search.apply(lambda row: 1 if (row['C1']==1)&(row['C2']==0) else 0, axis=1)
        X_Search['D3'] = X_Search.apply(lambda row: 1 if (row['C1']==0)&(row['C2']==1) else 0, axis=1)
        X_Search['D4'] = X_Search.apply(lambda row: 1 if (row['C1']==0)&(row['C2']==0) else 0, axis=1)
        X_Search['E1'] = np.where(X_Search['D1']==X_Search['A'], 1, 0)
        X_Search['E2'] = np.where(X_Search['D2']==X_Search['A'], 1, 0)
        X_Search['E3'] = np.where(X_Search['D3']==X_Search['A'], 1, 0)
        X_Search['E4'] = np.where(X_Search['D4']==X_Search['A'], 1, 0)
        E_X_Search_1 = sum(X_Search['Y']*X_Search['E1'])*2/df.shape[0]
        E_X_Search_2 = sum(X_Search['Y']*X_Search['E2'])*2/df.shape[0]
        E_X_Search_3 = sum(X_Search['Y']*X_Search['E3'])*2/df.shape[0]
        E_X_Search_4 = sum(X_Search['Y']*X_Search['E4'])*2/df.shape[0]
        E = [E_X_Search_4,E_X_Search_3,E_X_Search_2,E_X_Search_1]
        index, value = max(enumerate(E), key=operator.itemgetter(1))
        print("x0 "+str(i)+" x3 "+str(j),value,index)

x0 1 x3 1 57.6648694 3
x0 1 x3 2 57.9854954 3
x0 1 x3 3 57.0129174 3
x0 1 x3 4 57.0129174 3
x0 1 x3 5 56.8355874 0
x0 2 x3 1 57.4341794 3
x0 2 x3 2 56.9004674 3
x0 2 x3 3 55.9278894 3
x0 2 x3 4 55.5853834 2
x0 2 x3 5 56.8355874 0
x0 3 x3 1 59.7325734 3
x0 3 x3 2 57.3042194 3
x0 3 x3 3 57.4180754 3
x0 3 x3 4 56.6770534 0
x0 3 x3 5 56.8355874 0
x0 4 x3 1 59.0946554 3
x0 4 x3 2 56.6663014 3
x0 4 x3 3 56.7801574 3
x0 4 x3 4 57.2379594 0
x0 4 x3 5 56.8355874 0
x0 5 x3 1 58.7674974 3
x0 5 x3 2 57.2304514 3
x0 5 x3 3 58.5263794 3
x0 5 x3 4 55.7505534 3
x0 5 x3 5 56.8355874 0
x0 6 x3 1 56.6193654 3
x0 6 x3 2 54.8829814 3
x0 6 x3 3 57.3463734 3
x0 6 x3 4 53.9871174 0
x0 6 x3 5 56.8355874 0
x0 7 x3 1 55.4967434 3
x0 7 x3 2 57.5667894 1
x0 7 x3 3 53.6055194 0
x0 7 x3 4 57.3868094 0
x0 7 x3 5 56.8355874 0
x0 8 x3 1 55.6521794 1
x0 8 x3 2 56.5391474 1
x0 8 x3 3 55.6471174 0
x0 8 x3 4 59.0158754 0
x0 8 x3 5 56.8355874 0
x0 9 x3 1 60.4943514 0
x0 9 x3 2 59.6073834 0
x0 9 x3 3 58.4540374 0
x0 9 x3 4 5

### Case 5. var1 and var7 (Cont 2 and Nom 2)

In [17]:
for i in unique_X1:
    for j in range(len(sets_X7)):
        X_Search = df.copy()[['Cont2','Nom2','A','Y']];
        X_Search['C1'] = df['Cont2'].apply(lambda x : 1 if x<i else 0)
        X_Search['C2'] = df['Nom2'].apply(lambda x : 1 if x in sets_X7[j] else 0)
        X_Search['D1'] = X_Search.apply(lambda row: 1 if (row['C1']==1)&(row['C2']==1) else 0, axis=1)
        X_Search['D2'] = X_Search.apply(lambda row: 1 if (row['C1']==1)&(row['C2']==0) else 0, axis=1)
        X_Search['D3'] = X_Search.apply(lambda row: 1 if (row['C1']==0)&(row['C2']==1) else 0, axis=1)
        X_Search['D4'] = X_Search.apply(lambda row: 1 if (row['C1']==0)&(row['C2']==0) else 0, axis=1)
        X_Search['E1'] = np.where(X_Search['D1']==X_Search['A'], 1, 0)
        X_Search['E2'] = np.where(X_Search['D2']==X_Search['A'], 1, 0)
        X_Search['E3'] = np.where(X_Search['D3']==X_Search['A'], 1, 0)
        X_Search['E4'] = np.where(X_Search['D4']==X_Search['A'], 1, 0)
        E_X_Search_1 = sum(X_Search['Y']*X_Search['E1'])*2/df.shape[0]
        E_X_Search_2 = sum(X_Search['Y']*X_Search['E2'])*2/df.shape[0]
        E_X_Search_3 = sum(X_Search['Y']*X_Search['E3'])*2/df.shape[0]
        E_X_Search_4 = sum(X_Search['Y']*X_Search['E4'])*2/df.shape[0]
        E = [E_X_Search_4,E_X_Search_3,E_X_Search_2,E_X_Search_1]
        index, value = max(enumerate(E), key=operator.itemgetter(1))
        print("x1 "+str(i)+" x7 "+str(j),value,index)

x1 1 x7 0 58.7514954 2
x1 1 x7 1 58.5519354 3
x1 1 x7 2 59.0502974 3
x1 1 x7 3 60.7666454 3
x1 1 x7 4 59.6640214 2
x1 1 x7 5 57.9476734 2
x1 1 x7 6 58.1377714 3
x1 1 x7 7 60.9987754 2
x1 1 x7 8 59.2824274 2
x1 1 x7 9 58.7840654 2
x1 1 x7 10 61.9113014 2
x1 1 x7 11 57.9802434 3
x1 1 x7 12 59.6965914 3
x1 1 x7 13 60.1949534 3
x1 1 x7 14 58.5193654 2
x1 1 x7 15 59.8541194 2
x1 2 x7 0 59.0625954 2
x1 2 x7 1 59.9901774 3
x1 2 x7 2 59.5971854 3
x1 2 x7 3 62.7517754 3
x1 2 x7 4 59.9751214 2
x1 2 x7 5 59.0776514 3
x1 2 x7 6 58.6846594 3
x1 2 x7 7 62.6553474 2
x1 2 x7 8 59.5007574 2
x1 2 x7 9 59.8937494 2
x1 2 x7 10 63.5678734 2
x1 2 x7 11 58.2464974 2
x1 2 x7 12 60.8062754 3
x1 2 x7 13 60.4132834 3
x1 2 x7 14 59.1590234 2
x1 2 x7 15 61.8392494 2
x1 3 x7 0 57.7005314 2
x1 3 x7 1 57.6443334 2
x1 3 x7 2 59.2373554 3
x1 3 x7 3 59.2935534 3
x1 3 x7 4 59.9420734 2
x1 3 x7 5 59.8858754 2
x1 3 x7 6 57.5403054 2
x1 3 x7 7 59.8282894 2
x1 3 x7 8 59.7720914 2
x1 3 x7 9 57.4265214 2
x1 3 x7 10 62.0698314 

### Case 6. var5 and var8 (Ord 3 and Nom 3)

In [18]:
for i in unique_X5:
    for j in range(len(sets_X8)):
        X_Search = df.copy()[['Ord3','Nom3','A','Y']];
        X_Search['C1'] = df['Ord3'].apply(lambda x : 1 if x<i else 0)
        X_Search['C2'] = df['Nom3'].apply(lambda x : 1 if x in sets_X7[j] else 0)
        X_Search['D1'] = X_Search.apply(lambda row: 1 if (row['C1']==1)&(row['C2']==1) else 0, axis=1)
        X_Search['D2'] = X_Search.apply(lambda row: 1 if (row['C1']==1)&(row['C2']==0) else 0, axis=1)
        X_Search['D3'] = X_Search.apply(lambda row: 1 if (row['C1']==0)&(row['C2']==1) else 0, axis=1)
        X_Search['D4'] = X_Search.apply(lambda row: 1 if (row['C1']==0)&(row['C2']==0) else 0, axis=1)
        X_Search['E1'] = np.where(X_Search['D1']==X_Search['A'], 1, 0)
        X_Search['E2'] = np.where(X_Search['D2']==X_Search['A'], 1, 0)
        X_Search['E3'] = np.where(X_Search['D3']==X_Search['A'], 1, 0)
        X_Search['E4'] = np.where(X_Search['D4']==X_Search['A'], 1, 0)
        E_X_Search_1 = sum(X_Search['Y']*X_Search['E1'])*2/df.shape[0]
        E_X_Search_2 = sum(X_Search['Y']*X_Search['E2'])*2/df.shape[0]
        E_X_Search_3 = sum(X_Search['Y']*X_Search['E3'])*2/df.shape[0]
        E_X_Search_4 = sum(X_Search['Y']*X_Search['E4'])*2/df.shape[0]
        E = [E_X_Search_4,E_X_Search_3,E_X_Search_2,E_X_Search_1]
        index, value = max(enumerate(E), key=operator.itemgetter(1))
        print("x5 "+str(i)+" x8 "+str(j),value,index)

x5 1 x8 0 58.0366794 2
x5 1 x8 1 57.8621374 2
x5 1 x8 2 59.0214434 3
x5 1 x8 3 59.1959854 3
x5 1 x8 4 57.6123654 3
x5 1 x8 5 57.7869074 3
x5 1 x8 6 59.7982214 3
x5 1 x8 7 59.5566934 2
x5 1 x8 8 59.3821514 2
x5 1 x8 9 57.5014294 3
x5 1 x8 10 58.7799154 2
x5 1 x8 11 58.4527494 2
x5 1 x8 12 58.2782074 2
x5 1 x8 13 58.6053734 3
x5 1 x8 14 57.6759714 2
x5 1 x8 15 59.9727634 2
x5 2 x8 0 56.8355874 1
x5 2 x8 1 55.4220354 3
x5 2 x8 2 59.9575214 1
x5 2 x8 3 55.4930754 1
x5 2 x8 4 52.4219834 3
x5 2 x8 5 53.8409854 0
x5 2 x8 6 54.652078 1
x5 2 x8 7 55.5080074 3
x5 2 x8 8 54.0944554 3
x5 2 x8 9 56.9927874 1
x5 2 x8 10 52.6280394 2
x5 2 x8 11 55.3203814 3
x5 2 x8 12 53.9068294 3
x5 2 x8 13 56.8057194 1
x5 2 x8 14 52.8156654 2
x5 2 x8 15 53.9928014 3
x5 3 x8 0 56.8355874 1
x5 3 x8 1 54.0410294 1
x5 3 x8 2 58.7536434 3
x5 3 x8 3 55.6702034 3
x5 3 x8 4 53.6305674 2
x5 3 x8 5 56.7140074 2
x5 3 x8 6 53.387616 1
x5 3 x8 7 57.2180714 3
x5 3 x8 8 54.1346314 3
x5 3 x8 9 59.1361274 3
x5 3 x8 10 53.2480834 2
